In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load and process a single-chain or multi-chain protein from a PDB file

In [46]:
from byprot.datamodules.datasets import DataProcessor

In [58]:
pdb_path = "/root/research/projects/ByProt_public/examples/3f4m.pdb"
pdb_path = "/root/research/projects/ByProt_public/examples/3uat.pdb"

In [67]:
dp = DataProcessor()
structure = dp.parse_PDB(
    pdb_path,
    # input_chain_list=['A', 'B'] -> load which chains
    # masked_chain_list=['A']     -> which chains to predict while the remaining chains serve as conditioning
)

In [66]:
print(structure.keys())
print(structure)

dict_keys(['seq_chain_A', 'coords_chain_A', 'seq_chain_B', 'coords_chain_B', 'name', 'num_of_chains', 'seq', 'coords', 'masked_list', 'visible_list'])


#### single-chain protein 

In [ ]:
from byprot.datamodules.datasets.data_utils import Alphabet
alphabet = Alphabet('esm', 'cath')
alphabet.featurize([structure])

#### multi-chain protein

In [ ]:
alphabet = Alphabet('esm', 'multichain')
alphabet.featurize([structure])

## Design sequences for structures

In [ ]:
from byprot.utils.config import compose_config as Cfg
from byprot.tasks.fixedbb.designer import Designer

# 1. instantialize designer
cfg = Cfg(
    cuda=True,
    generator=Cfg(
        max_iter=5,
        strategy='denoise',
        temperature=0,
        eval_sc=False,
    )
)
exp_path = "/root/research/projects/ByProt_public/logs/cath4.2/lm_design_esm2_650m"
designer_cath = Designer(experiment_path=exp_path, cfg=cfg)

exp_path = "/root/research/projects/ByProt_public/logs/fixedbb_multichain/lm_design_esm2_650m"
designer_complex = Designer(experiment_path=exp_path, cfg=cfg)

In [ ]:
# multi-chain complex
pdb_path = "/root/research/projects/ByProt_public/examples/3uat.pdb"

print(f"designed by cath-trained LM-Design")
designer_cath.set_structure(pdb_path)
print(designer_cath.generate()[0]); designer_cath.calculate_metrics()

print(f"designed by pdb complex-trained LM-Design")
designer_complex.set_structure(pdb_path)
print(designer_complex.generate()[0]); designer_complex.calculate_metrics()

In [ ]:
# single chain
pdb_path = "/root/research/projects/ByProt_public/examples/3f4m.pdb"

print(f"designed by cath-trained LM-Design")
designer_cath.set_structure(pdb_path)
print(designer_cath.generate()[0]); designer_cath.calculate_metrics()

print(f"designed by pdb complex-trained LM-Design")
designer_complex.set_structure(pdb_path)
print(designer_complex.generate()[0]); designer_complex.calculate_metrics()